In [2]:
import pandas as pd
import os
import re
from unicodedata import normalize
import openpyxl

In [87]:
test_path = "./data/6_csv/final_data.xlsx"
ref_path = "./data/02_ref_db/ref.xlsx"

test_df = pd.read_excel(test_path,
                        index_col=0)
ref_df = pd.read_excel(ref_path)

In [88]:
# 파일 저장 중 생기는 유니코드 변경 방지
test_df["한자"] = test_df["한자"].apply(lambda x : normalize("NFKC", x))
ref_df["한자"] = ref_df["한자"].apply(lambda x : normalize("NFKC", x))

In [89]:
test_df

,출제연도,출제월,과목,지문번호,문제번호,한자
0,2023,11,국어,1,1,持
1,2023,11,국어,1,1,冊
2,2023,11,국어,1,1,理
3,2023,11,국어,1,1,中
4,2023,11,국어,1,1,書
...,...,...,...,...,...,...
50506,2026,9,국어,15,56,反
50507,2026,9,국어,15,56,映
50508,2026,9,국어,15,56,冊
50509,2026,9,국어,15,56,揷


In [90]:
ref_df

,한자,등급(대한검정회),등급(한국어문회),등급(기초교육용)
0,嫁,1.0,1.0,심화
1,恪,1.0,1.0,심화
2,侃,1.0,0.0,심화
3,玕,1.0,0.0,심화
4,揀,1.0,1.0,심화
...,...,...,...,...
2150,兔,0.0,3.5,심화
2151,姉,0.0,4.0,심화
2152,氷,0.0,5.0,중등
2153,強,0.0,6.0,심화


In [40]:
test_df["한자"].value_counts()

한자
對    1006
人     785
學     588
國     563
定     535
     ... 
憐       1
冒       1
擄       1
寡       1
珠       1
Name: count, Length: 1279, dtype: int64

In [82]:
result = pd.DataFrame.merge(
    pd.DataFrame(test_df["한자"].value_counts()),
    ref_df,
    how="left",
    on="한자"
)

"""
target_hanja = "理"

#print(len(result))
dd_power = result[result["등급(대한검정회)"].isnull()].loc[177, "한자"]
print(dd_power == target_hanja)
print(dd_power, ord(dd_power))
print(target_hanja, ord(target_hanja))
# dd_power[dd_power["한자"] == target_hanja]
"""
result[result["count"] > 20]


,한자,count,등급(대한검정회),등급(한국어문회),등급(기초교육용)
0,對,1006,5.0,6.5,중등
1,人,785,8.0,8.0,중등
2,學,588,5.0,8.0,중등
3,理,573,5.0,6.5,중등
4,國,563,5.0,8.0,중등
...,...,...,...,...,...
415,守,21,4.0,4.5,중등
416,暗,21,4.0,4.5,심화
417,印,21,3.5,4.5,심화
418,替,21,2.5,3.0,고등


,한자,등급(대한검정회),등급(한국어문회),등급(기초교육용)
1226,巖,3.0,3.5,심화


In [34]:
data = pd.DataFrame.merge(
    test_df,
    ref_df,
    how="left",
    on="한자"
)
data[data["등급(대한검정회)"].isnull()]

,출제연도,출제월,과목,지문번호,문제번호,한자,등급(대한검정회),등급(한국어문회),등급(기초교육용)
447,2023,11,국어,2,4,瀷,NaN,NaN,NaN
529,2023,11,국어,2,4,瀷,NaN,NaN,NaN
533,2023,11,국어,2,4,僿,NaN,NaN,NaN
607,2023,11,국어,2,4,僿,NaN,NaN,NaN
610,2023,11,국어,2,4,瀷,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
47592,2026,9,국어,8,31,令,NaN,NaN,NaN
47596,2026,9,국어,8,31,令,NaN,NaN,NaN
47699,2026,9,국어,8,31,廉,NaN,NaN,NaN
47820,2026,9,국어,9,33,來,NaN,NaN,NaN


#### 주제 : 현재 정의된 상용한자는 국어 중등교육용으로써 적합한가?
* 단순히 나온 횟수를 기반으로, 적절성을 파악한다.
* 서로 각기 다른 지문에 나온 한자 횟수를 기반으로, 적절성을 파악한다.
* 과목별 가중치를 두어, 적절성을 파악한다. (이거... 가중치 조절이 되면 재밌겠는데?)
* 과목별로, 적절성을 파악한다.
* 기초교육용 한자에 해당하는 한자의 비율을 계산하여, 적절성을 파악한다.
* 대한검정회 및 한국어문회 한자 급수에 해당하는 한자의 비율을 계산하여, 적절성을 파악한다.

* 디폴트는 각 교육기관마다의 적합성.
1. 단순횟수 기반
2. 가중치 기반 / 시험 응시율에 따라
3. 가중치 기반 / 평균적인 문제수에 따라
4. 각 과목별 기반
* 참고로, 가중치식은 w_1 (국어 지문과 그의 문제) + w_2 (사탐 과목별 문제) + w_3 (한국사 문제) 가 될 것으로 예상.